# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [2]:
def getNGrams(words,n):
    ngrams = {}
    for word in words:
        # word="$"*(n-1)+word+"$"*(n-1)
        if len(word) <n:
            continue
        for i in range(len(word)-n+1):
            ngram = word[i:i+n]
            if ngram not in ngrams:
                ngrams[ngram] = 1
            else:
                ngrams[ngram]+=1
    return ngrams


In [3]:
word=["abc"]
# guessed=list(set(word)-set("_"))
print(getNGrams(word,1))

{'a': 1, 'b': 1, 'c': 1}


In [4]:
HANGMAN_URL = "https://www.trexsim.com/trexsim/hangman"

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        # full_dictionary_location = "words_250000_train.txt"
        # self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        # self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        dictionary_file_location="words_250000_train.txt"
        text_file = open(dictionary_file_location,"r")
        words = text_file.read().splitlines()
        text_file.close()
        self.ngrams=[getNGrams(words,i) for i in range(1,6)]

        # self.current_dictionary = []
    def buildDict(self):
        dictionary_file_location="words_250000_train.txt"
        text_file = open(dictionary_file_location,"r")
        words = text_file.read().splitlines()
        text_file.close()
        
        self.ngrams=[getNGrams(words,i) for i in range(1,7)]
        
    def ngramprob(self,word,letter ):
        # print("________________________: word:",word,"letter:",letter)
        denominator=0
        underscroll_location=word.find("_")
        # if len(word)>2:
        #     print("word:",word,"letter:",letter)
        #     for l in string.ascii_lowercase:
        #         print("l:",l)
        #         print(self.ngrams[len(word)-1][word[:underscroll_location]+l+word[underscroll_location+1:]])

        for l in string.ascii_lowercase:
            
            try:
                denominator+=self.ngrams[len(word)-1][word[:underscroll_location]+l+word[underscroll_location+1:]]
            except:
                pass
        # print("____________:",word)
        # if len(word)>3:
        #     print("word with length >3:",word)
        # if word=="a_s":
        #     print("letter:",letter)
        #     print("denominator:",denominator)
        #     print("numerator:",self.ngrams[len(word)-1][word[:underscroll_location]+letter+word[underscroll_location+1:]])
        return self.ngrams[len(word)-1][word[:underscroll_location]+letter+word[underscroll_location+1:]]/denominator

        
    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        # clean_word = word[::2].replace("_",".")
        # guessed=list(set(word)-set("_"))
        word=word[::2]
        available = list(set(string.ascii_lowercase) - set(self.guessed_letters))
        probs=[]
        contributions=[0,0,0,0,0,0]
        for char in available:
            char_prob=0
            for i in range(len(word)):
                if word[i]!="_":
                    continue
                # find the longest continious string without any "_"

                # 4gram
                try:
                    if word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_":
                        contributions[3]+=self.ngramprob(word[i-3:i+1],char)
                        char_prob+=self.ngramprob(word[i-3:i+1],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i-1]!="_" and word[i-2]!="_":
                        contributions[3]+=self.ngramprob(word[i-2:i+2],char)
                        char_prob+=self.ngramprob(word[i-2:i+2],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i+2]!="_" and word[i-1]!="_":
                        contributions[3]+=self.ngramprob(word[i-1:i+3],char)
                        char_prob+=self.ngramprob(word[i-1:i+3],char)
                except:
                    pass
                try: 
                    if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_":
                        contributions[3]+=self.ngramprob(word[i:i+4],char)
                        char_prob+=self.ngramprob(word[i:i+4],char) 
                except:
                    pass
                # 3gram
                try:
                    if word[i-1]!="_" and word[i-2]!="_":
                        contributions[2]+=self.ngramprob(word[i-2:i+1],char)
                        char_prob+=self.ngramprob(word[i-2:i+1],char)
                except:
                    pass
                try:
                    # a_b
                    if word[i+1]!="_" and word[i-1]!="_":
                        contributions[2]+=self.ngramprob(word[i-1:i+2],char)
                        char_prob+=self.ngramprob(word[i-1:i+2],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i+2]!="_":
                        contributions[2]+=self.ngramprob(word[i:i+3],char)
                        char_prob+=self.ngramprob(word[i:i+3],char)
                except:
                    pass
                    # 2gram
                try:
                    if word[i-1]!="_":
                        contributions[1]+=self.ngramprob(word[i-1:i+1],char)
                        char_prob+=self.ngramprob(word[i-1:i+1],char)
                except:
                    pass
                try:
                    if word[i+1]!="_":
                        contributions[1]+=self.ngramprob(word[i:i+2],char)
                        char_prob+=self.ngramprob(word[i:i+2],char)
                except:
                    pass
                # 5 gram
                try:
                    if word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_" and word[i-4]!="_" :
                        contributions[4]+=self.ngramprob(word[i-4:i+1],char)
                        char_prob+=self.ngramprob(word[i-4:i+1],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_" :
                        contributions[4]+=self.ngramprob(word[i-3:i+2],char)
                        char_prob+=self.ngramprob(word[i-3:i+2],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i+2]!="_" and word[i-1]!="_" and word[i-2]!="_" :
                        contributions[4]+=self.ngramprob(word[i-2:i+3],char)
                        char_prob+=self.ngramprob(word[i-2:i+3],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_" and word[i-1]!="_" :
                        contributions[4]+=self.ngramprob(word[i-1:i+4],char)
                        char_prob+=self.ngramprob(word[i-1:i+4],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_" and word[i+4]!="_" :
                        contributions[4]+=self.ngramprob(word[i:i+5],char)
                        char_prob+=self.ngramprob(word[i:i+5],char)
                except:
                    pass
                # 6 gram
                try:
                    if word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_" and word[i-4]!="_" and word[i-5]!="_" :
                        contributions[5]+=self.ngramprob(word[i-5:i+1],char)
                        char_prob+=self.ngramprob(word[i-5:i+1],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_" and word[i-4]!="_" :
                        contributions[5]+=self.ngramprob(word[i-4:i+2],char)
                        char_prob+=self.ngramprob(word[i-4:i+2],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i+2]!="_" and word[i-1]!="_" and word[i-2]!="_" and word[i-3]!="_" :
                        contributions[5]+=self.ngramprob(word[i-3:i+3],char)
                        char_prob+=self.ngramprob(word[i-3:i+3],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_" and word[i-1]!="_" and word[i-2]!="_" :
                        contributions[5]+=self.ngramprob(word[i-2:i+4],char)
                        char_prob+=self.ngramprob(word[i-2:i+4],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_" and word[i+4]!="_" and word[i-1]!="_" :
                        contributions[5]+=self.ngramprob(word[i-1:i+5],char)
                        char_prob+=self.ngramprob(word[i-1:i+5],char)
                except:
                    pass
                try:
                    if word[i+1]!="_" and word[i+2]!="_" and word[i+3]!="_" and word[i+4]!="_" and word[i+5]!="_" :
                        contributions[5]+=self.ngramprob(word[i:i+6],char)
                        char_prob+=self.ngramprob(word[i:i+6],char)
                except:
                    pass


                # 1gram
                char_prob+=self.ngramprob(word[i:i+1],char)
                contributions[0]+=self.ngramprob(word[i:i+1],char)
            probs.append(char_prob)
        # create sorted dictionary of probabilities
        probs_dict=dict(zip(available,probs))
        probs_sorted=sorted(probs_dict.items(), key=lambda x: x[1], reverse=True)
        # iterate on probs_sorted
        for i in probs_sorted:
            # round the probability to 3 decimal places
            # i[1]=round()
            print(i[0],round(i[1],2),end=", ")
        # for i in range(len(probs)):
        #     # print upto 2 decimal places
        #     # probs[i]=round(probs[i],2)
        #     print(probs_sorted[i],round(probs_sorted[i],2),end=" ")
        print()
        print("contributions:",contributions)
        return available[probs.index(max(probs))]
            
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        # self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    def offline(self,wordtobeguessed):
        self.guessed_letters=[]
        tries_remains=6
        word = "_ " * len(wordtobeguessed)
        assert len(word) == len(wordtobeguessed) *2
        while tries_remains>0:
                # get guessed letter from user code
            guess_letter = self.guess(word)
            flag=False
            for index in range(len(wordtobeguessed)):
                if wordtobeguessed[index]==guess_letter:
                    word=word[:2*index]+guess_letter+word[2*index+1:]
                    flag=True
            if flag==False:
                tries_remains-=1
                print("Wrong guess, you have",tries_remains,"tries left")
            if "_" not in word:
                print("You won!")
                break
            # append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            # time.sleep(0.5)
            print("Guessed letter: {0}".format(guess_letter))
            print("Current word: {0}".format(word))
            
            # tries_remains -=1
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        num_retry, time_sleep = 5, 2                                                                                        
        for it in range(num_retry):                                                                                         
            try:                                                                                                            
                response = self.session.request(                                                                            
                    method or "GET",                                                                                        
                    HANGMAN_URL + path,                                                                                     
                    timeout=self.timeout,                                                                                   
                    params=args,                                                                                            
                    data=post_args                                                                                          
                )                                                                                                           
                break                                                                                                       
            except requests.HTTPError as e:                                                                                 
                response = json.loads(e.read())                                                                             
                raise HangmanAPIError(response)                                                                             
            except requests.exceptions.SSLError as e:                                                                       
                if it + 1 == num_retry:                                                                                     
                    raise                                                                                                   
                time.sleep(time_sleep)  

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [10]:
api = HangmanAPI(access_token="1351d29da90b3d587ad0f8b1720f69", timeout=2000)


In [11]:
api.offline("cancelled")

e 0.99, i 0.78, a 0.76, n 0.64, o 0.64, r 0.63, s 0.63, t 0.58, l 0.52, c 0.38, u 0.33, d 0.32, p 0.28, m 0.26, h 0.25, g 0.22, y 0.17, b 0.17, f 0.11, v 0.09, k 0.08, w 0.08, z 0.04, x 0.03, q 0.02, j 0.02, 
contributions: [9.000000000000004, 0, 0, 0, 0, 0]
Guessed letter: e
Current word: _ _ _ _ e _ _ e _ 
r 1.1, n 0.91, s 0.88, t 0.79, l 0.74, a 0.72, i 0.71, d 0.6, o 0.54, c 0.45, m 0.38, p 0.36, h 0.31, u 0.3, g 0.26, b 0.21, v 0.19, y 0.16, f 0.14, k 0.11, w 0.1, z 0.06, x 0.05, j 0.02, q 0.02, 
contributions: [6.229924423907603, 3.8976370999817793, 0, 0, 0, 0]
Wrong guess, you have 5 tries left
Guessed letter: r
Current word: _ _ _ _ e _ _ e _ 
n 0.91, s 0.88, t 0.79, l 0.74, a 0.72, i 0.71, d 0.6, o 0.54, c 0.45, m 0.38, p 0.36, h 0.31, u 0.3, g 0.26, b 0.21, v 0.19, y 0.16, f 0.14, k 0.11, w 0.1, z 0.06, x 0.05, j 0.02, q 0.02, 
contributions: [5.7382910710268344, 3.285836232341359, 0, 0, 0, 0]
Guessed letter: n
Current word: _ _ n _ e _ _ e _ 
t 1.07, i 1.01, s 0.94, a 0.91, 

In [112]:
# words=["ab","bc"]

# print(ngrams)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [7]:
[total_practice_runs_,total_recorded_runs,total_recorded_successes,total_practice_successes_] = api.my_status()
for _ in range(100):
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    # practice_success_rate = total_practice_successes / total_practice_runs
    print('practice success rate so far = ' , (total_practice_successes-total_practice_successes_)/(total_practice_runs-total_practice_runs_))
print(total_practice_successes-total_practice_successes_)

Successfully start a new game! Game ID: 38876a3b911f. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ .
e 1.1, i 0.87, a 0.85, n 0.72, o 0.71, r 0.7, s 0.7, t 0.65, l 0.58, c 0.42, u 0.36, d 0.35, p 0.31, m 0.29, h 0.27, g 0.24, y 0.19, b 0.19, f 0.12, v 0.1, k 0.09, w 0.08, z 0.04, x 0.03, q 0.02, j 0.02, 
contributions: [10.000000000000009, 0, 0, 0, 0, 0]
Guessing letter: e
Sever response: {'game_id': '38876a3b911f', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ e _ _ _ _ _ e '}
r 0.98, n 0.86, s 0.82, t 0.8, i 0.78, a 0.76, l 0.73, o 0.6, d 0.52, c 0.46, m 0.37, p 0.36, h 0.33, u 0.32, g 0.27, b 0.21, v 0.19, y 0.17, f 0.14, k 0.12, w 0.1, z 0.07, x 0.04, j 0.02, q 0.02, 
contributions: [7.119913627322973, 2.9247562523208437, 0, 0, 0, 0]
Guessing letter: r
Sever response: {'game_id': '38876a3b911f', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ e _ _ _ _ _ e '}
n 0.86, s 0.82, t 0.8, i 0.78, a 0.76, l 0.73, o 0.6, d 0.52, c 0.46, m 0.37, p 0.36, h 0.33, u 0.32

KeyboardInterrupt: 

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [23]:
print(api.my_status()[2]/api.my_status()[1])
print(api.my_status())

0.528
[813, 1000, 528, 327]


In [22]:
[total_practice_runs_,total_recorded_runs_,total_recorded_successes_,total_practice_successes_] = api.my_status()

for i in range(8):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    print("Successes so far:", total_recorded_successes-total_recorded_successes_)
    print('practice success rate so far = ' , (total_recorded_successes-total_recorded_successes_)/(total_recorded_runs-total_recorded_runs_))

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Successfully start a new game! Game ID: 4d391b45fd82. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
e 0.99, i 0.78, a 0.76, n 0.64, o 0.64, r 0.63, s 0.63, t 0.58, l 0.52, c 0.38, u 0.33, d 0.32, p 0.28, m 0.26, h 0.25, g 0.22, y 0.17, b 0.17, f 0.11, v 0.09, k 0.08, w 0.08, z 0.04, x 0.03, q 0.02, j 0.02, 
contributions: [9.000000000000004, 0, 0, 0, 0, 0]
Guessing letter: e
Sever response: {'game_id': '4d391b45fd82', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ _ _ e _ e _ '}
r 1.51, n 1.23, s 1.05, t 0.99, l 0.97, d 0.79, a 0.73, i 0.68, c 0.51, m 0.5, o 0.49, p 0.42, h 0.35, g 0.3, v 0.29, u 0.29, b 0.25, y 0.18, f 0.17, k 0.14, w 0.13, x 0.08, z 0.08, j 0.03, q 0.02, 
contributions: [5.339935220492233, 5.846455649972673, 0.9995962392570802, 0, 0, 0]
Guessing letter: r
Sever response: {'game_id': '4d391b45fd82', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e r r _ e r e _ '}
i 3.94, t 1.89, d 1.59, n 1.08, s 0.99, a 0.98, h 0.97, o 0.78, p 

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [21]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.516
